<a href="https://colab.research.google.com/github/akshala/Assembler/blob/master/logisticRegression_FastText.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from nltk.tokenize import sent_tokenize
from nltk.tokenize import TreebankWordTokenizer
from scipy.stats.stats import pearsonr
from scipy.stats.stats import spearmanr
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from gensim.models import FastText
import matplotlib.cm as cm
from tempfile import TemporaryFile

In [2]:
from google.colab import files
uploaded = files.upload()

Saving each_day_label_text.csv to each_day_label_text.csv


In [3]:
import pandas as pd
df = pd.read_csv('each_day_label_text.csv')
df = df.dropna()
# df

In [4]:
df['label'].loc[(df['label'] == 'Shock')] = 1
df['label'].loc[(df['label'] == 'Non-Shock')] = 0
# df

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [5]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
stop_words = set(stopwords.words('english'))
punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''

In [7]:
#Making list of sentences after
sentences = []
for text in df["text"]:
   #1. Replace "\n" with " " (spaces)
   text = text.replace("\n", " ")
   text = text.replace("\t", " ")
   #2. Replace "[** - **]" data in this format with "" (mostly time and date)
   text = re.sub(r'\[\*\*.*?\*\*\]', "", text)
   #3. Removed wide spaces
   text = re.sub(' +', " ", text)
   sentences.append(sent_tokenize(text))

In [8]:
words = []
for sen in sentences:
    patient_words = []
    for s in sen :
        s_sep = ""
        temp = []
        for character in s:
            if character in punctuations:
                outputCharacter = " %s " % character
            else:
                outputCharacter = character
            s_sep += outputCharacter
        # Tokenize sentences to word tokens
        for wrd in TreebankWordTokenizer().tokenize(s_sep):
            # Removed stop words and punctations
            if wrd not in punctuations:
                #6. Converted all the words to lower case
                wrd_lower = wrd.lower()
                #wrd_lower = lancaster.stem(wrd_lower)
                if ((wrd_lower not in stop_words) & (not wrd_lower.isdigit()) & (len(wrd_lower)>1) & (wrd_lower != '\ufeff' )):
                    temp.append(wrd_lower)
        patient_words.extend(temp)
    words.append(patient_words)

In [ ]:
from gensim.models import FastText
model = FastText(words, min_count = 1, size = 300, window = 5, sg = 1)

In [ ]:
model['sepsis']

In [ ]:
def sif_embeddings(sentences, model, alpha=1e-3):
    """Compute the SIF embeddings for a list of sentences
    Parameters
    ----------
    sentences : list
        The sentences to compute the embeddings for
    model : `~gensim.models.base_any2vec.BaseAny2VecModel`
        A gensim model that contains the word vectors and the vocabulary
    alpha : float, optional
        Parameter which is used to weigh each individual word based on its probability p(w).
    Returns
    -------
    numpy.ndarray 
        SIF sentence embedding matrix of dim len(sentences) * dimension
    """
    
    vlookup = model.wv.vocab  # Gives us access to word index and count
    vectors = model.wv        # Gives us access to word vectors
    size = model.vector_size  # Embedding size
    
    Z = 0
    for k in vlookup:
        Z += vlookup[k].count # Compute the normalization constant Z
    
    output = []
    
    # Iterate all sentences
    for s in sentences:
        count = 0
        v = np.zeros(size, dtype=REAL) # Summary vector
        # Iterare all words
        for w in s:
            # A word must be present in the vocabulary
            if w in vlookup:
                for i in range(size):
                    v[i] += ( alpha / (alpha + (vlookup[w].count / Z))) * vectors[w][i]
                count += 1 
                
        if count > 0:
            for i in range(size):
                v[i] *= 1/count
        output.append(v)
    return np.vstack(output).astype(REAL)

In [ ]:
REAL = np.float32 
array = sif_embeddings(words, model, 1e-3)
array.shape

(2569, 300)

In [ ]:
array

array([[-0.03017591,  0.03076372, -0.03969172, ..., -0.00711262,
        -0.06732748,  0.02095205],
       [-0.00549397,  0.01410158, -0.05221346, ...,  0.01477716,
        -0.06164019,  0.02998966],
       [-0.01308691,  0.01827761, -0.05502955, ...,  0.00611987,
        -0.06180539,  0.0424997 ],
       ...,
       [-0.03443385,  0.05112674, -0.05015321, ..., -0.01269269,
        -0.06895726,  0.04280884],
       [-0.01794074,  0.0547568 , -0.03875435, ..., -0.00679507,
        -0.05527372,  0.03846744],
       [-0.0308524 ,  0.06115716, -0.05419201, ..., -0.00519136,
        -0.06308854,  0.05395428]], dtype=float32)

In [ ]:
np.savetxt('FastText_embeddings.txt', array)

In [10]:
from google.colab import files
uploaded = files.upload()

Saving FastText_embeddings.txt to FastText_embeddings.txt


In [11]:
array = np.loadtxt('FastText_embeddings.txt')
array

array([[-0.02792864,  0.04547333, -0.02749095, ..., -0.01844831,
        -0.07049414,  0.02148656],
       [-0.00413048,  0.02610454, -0.04540325, ...,  0.00562713,
        -0.07395396,  0.0368084 ],
       [-0.01384597,  0.03443255, -0.04860777, ..., -0.00159635,
        -0.07496523,  0.04420214],
       ...,
       [-0.03404125,  0.06368995, -0.0450493 , ..., -0.02192115,
        -0.07986488,  0.04349259],
       [-0.0221134 ,  0.06743413, -0.03308356, ..., -0.01464292,
        -0.06522159,  0.0383691 ],
       [-0.03598973,  0.07643144, -0.0546134 , ..., -0.014615  ,
        -0.06654139,  0.05352054]])

In [12]:
n = len(df)
embeddings = {}
for i in range(n):
  embeddings[i] = array[i]

In [13]:
embedding_df = pd.DataFrame.from_dict(embeddings, orient = 'index')
embedding_df['label'] = df['label']
embedding_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,label
0,-0.027929,0.045473,-0.027491,0.015190,0.000107,-0.024935,-0.032980,0.008431,-0.037732,0.030084,0.016702,0.064239,0.014436,-0.027693,-0.007810,-0.029422,-0.003710,0.021838,0.023579,0.016537,-0.014005,0.022275,-0.053586,-0.025039,0.019068,-0.040851,-0.054432,0.041027,0.003569,-0.010553,-0.023212,-0.005068,0.002998,0.005275,-0.024828,-0.000735,0.052682,0.013410,-0.021471,-0.032271,...,-0.036897,-0.046403,-0.028568,0.035028,0.004339,0.047216,-0.045766,0.015662,-0.069638,0.052229,0.006115,0.023211,0.008596,0.060139,-0.008416,-0.005664,-0.009244,-0.047376,0.047727,-0.004606,0.046259,0.067883,-0.008340,-0.041430,0.048779,-0.025553,0.047854,0.038176,0.034293,0.061943,0.019458,-0.014631,0.043594,-0.028582,-0.016275,-0.017250,-0.018448,-0.070494,0.021487,1
1,-0.004130,0.026105,-0.045403,-0.003086,0.004016,-0.022892,-0.008053,0.016219,-0.032984,0.034626,0.007679,0.042517,0.023115,-0.024550,-0.011162,-0.031245,-0.015645,0.022321,0.020165,0.021781,0.007559,0.028865,-0.060094,-0.006792,0.028574,-0.047876,-0.052358,0.037748,0.004507,-0.018644,-0.005876,0.000185,0.001869,-0.007142,-0.011379,0.000347,0.040178,0.003706,-0.020116,-0.039740,...,-0.059650,-0.049295,-0.017975,0.030011,-0.002585,0.020801,-0.046827,0.018261,-0.074041,0.046927,-0.012671,0.001476,-0.009282,0.049995,0.001530,-0.014517,-0.002215,-0.048543,0.034480,-0.025405,0.031593,0.054605,-0.000143,-0.048124,0.026710,-0.034582,0.042300,0.050427,0.047222,0.054603,0.032794,-0.015163,0.041301,-0.040048,0.001588,-0.010156,0.005627,-0.073954,0.036808,0
2,-0.013846,0.034433,-0.048608,-0.007782,-0.002638,-0.020973,-0.014225,0.014021,-0.033846,0.042294,0.012886,0.046607,0.025992,-0.029386,-0.005675,-0.030501,-0.017116,0.022658,0.024673,0.020325,0.009300,0.024187,-0.063379,-0.010718,0.021502,-0.056413,-0.058291,0.037070,0.004613,-0.014826,-0.015049,-0.001719,-0.000801,-0.002586,-0.017067,-0.004330,0.041612,0.005221,-0.030770,-0.035603,...,-0.064650,-0.045617,-0.023339,0.028378,-0.007444,0.022630,-0.050858,0.017568,-0.064044,0.047588,-0.021169,0.002682,-0.002921,0.067823,-0.000294,-0.013041,0.004509,-0.053449,0.036137,-0.020958,0.023003,0.057330,0.005377,-0.049994,0.028060,-0.040692,0.039361,0.047700,0.048854,0.058045,0.030292,-0.010244,0.043663,-0.041667,-0.007432,-0.012866,-0.001596,-0.074965,0.044202,1
3,-0.020380,0.062005,-0.036003,0.003629,-0.004798,-0.028179,-0.043848,0.003861,-0.049418,0.037786,0.000296,0.062605,0.017976,-0.018270,0.001802,-0.029008,0.004497,0.037384,0.032797,0.013666,-0.013678,0.005107,-0.052585,-0.032091,0.023455,-0.039506,-0.071952,0.041092,0.008768,-0.012441,-0.021543,-0.013440,-0.011511,0.004086,-0.016868,-0.003797,0.062168,0.021020,-0.017684,-0.033057,...,-0.036466,-0.048071,-0.044052,0.031582,-0.005607,0.046977,-0.046315,0.012168,-0.049222,0.055603,-0.008354,0.027261,0.011111,0.057126,0.008092,-0.002842,-0.010246,-0.037934,0.049191,-0.018213,0.045192,0.095404,-0.007928,-0.039156,0.048771,-0.034426,0.038492,0.039559,0.031308,0.052141,0.027275,-0.008246,0.043803,-0.032329,-0.021422,-0.018111,-0.009990,-0.060476,0.027701,1
4,-0.003644,0.028737,-0.052126,-0.000054,0.008107,-0.017344,-0.013791,0.012748,-0.035662,0.039845,0.006583,0.042427,0.022603,-0.026439,-0.011025,-0.032833,-0.010979,0.014916,0.026037,0.021582,0.004839,0.028031,-0.069568,-0.010970,0.035693,-0.046544,-0.054415,0.052412,0.012303,-0.015019,-0.008838,-0.008885,0.001337,-0.009247,-0.010716,-0.001443,0.034402,0.002758,-0.020472,-0.038950,...,-0.058288,-0.054549,-0.024948,0.033424,-0.003804,0.013218,-0.054962,0.012327,-0.076053,0.053361,-0.018637,0.001523,-0.005087,0.054736,-0.002094,-0.014366,0.003204,-0.057732,0.036873,-0.031804,0.027221,0.055413,0.007639,-0.042472,0.022434,-0.037340,0.041808,0.061506,0.053400,0.056493,0.034941,-0

In [14]:
X = embedding_df.drop(['label'], axis=1)
X.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
0,-0.027929,0.045473,-0.027491,0.015190,0.000107,-0.024935,-0.032980,0.008431,-0.037732,0.030084,0.016702,0.064239,0.014436,-0.027693,-0.007810,-0.029422,-0.003710,0.021838,0.023579,0.016537,-0.014005,0.022275,-0.053586,-0.025039,0.019068,-0.040851,-0.054432,0.041027,0.003569,-0.010553,-0.023212,-0.005068,0.002998,0.005275,-0.024828,-0.000735,0.052682,0.013410,-0.021471,-0.032271,...,0.051648,-0.036897,-0.046403,-0.028568,0.035028,0.004339,0.047216,-0.045766,0.015662,-0.069638,0.052229,0.006115,0.023211,0.008596,0.060139,-0.008416,-0.005664,-0.009244,-0.047376,0.047727,-0.004606,0.046259,0.067883,-0.008340,-0.041430,0.048779,-0.025553,0.047854,0.038176,0.034293,0.061943,0.019458,-0.014631,0.043594,-0.028582,-0.016275,-0.017250,-0.018448,-0.070494,0.021487
1,-0.004130,0.026105,-0.045403,-0.003086,0.004016,-0.022892,-0.008053,0.016219,-0.032984,0.034626,0.007679,0.042517,0.023115,-0.024550,-0.011162,-0.031245,-0.015645,0.022321,0.020165,0.021781,0.007559,0.028865,-0.060094,-0.006792,0.028574,-0.047876,-0.052358,0.037748,0.004507,-0.018644,-0.005876,0.000185,0.001869,-0.007142,-0.011379,0.000347,0.040178,0.003706,-0.020116,-0.039740,...,0.051488,-0.059650,-0.049295,-0.017975,0.030011,-0.002585,0.020801,-0.046827,0.018261,-0.074041,0.046927,-0.012671,0.001476,-0.009282,0.049995,0.001530,-0.014517,-0.002215,-0.048543,0.034480,-0.025405,0.031593,0.054605,-0.000143,-0.048124,0.026710,-0.034582,0.042300,0.050427,0.047222,0.054603,0.032794,-0.015163,0.041301,-0.040048,0.001588,-0.010156,0.005627,-0.073954,0.036808
2,-0.013846,0.034433,-0.048608,-0.007782,-0.002638,-0.020973,-0.014225,0.014021,-0.033846,0.042294,0.012886,0.046607,0.025992,-0.029386,-0.005675,-0.030501,-0.017116,0.022658,0.024673,0.020325,0.009300,0.024187,-0.063379,-0.010718,0.021502,-0.056413,-0.058291,0.037070,0.004613,-0.014826,-0.015049,-0.001719,-0.000801,-0.002586,-0.017067,-0.004330,0.041612,0.005221,-0.030770,-0.035603,...,0.050406,-0.064650,-0.045617,-0.023339,0.028378,-0.007444,0.022630,-0.050858,0.017568,-0.064044,0.047588,-0.021169,0.002682,-0.002921,0.067823,-0.000294,-0.013041,0.004509,-0.053449,0.036137,-0.020958,0.023003,0.057330,0.005377,-0.049994,0.028060,-0.040692,0.039361,0.047700,0.048854,0.058045,0.030292,-0.010244,0.043663,-0.041667,-0.007432,-0.012866,-0.001596,-0.074965,0.044202
3,-0.020380,0.062005,-0.036003,0.003629,-0.004798,-0.028179,-0.043848,0.003861,-0.049418,0.037786,0.000296,0.062605,0.017976,-0.018270,0.001802,-0.029008,0.004497,0.037384,0.032797,0.013666,-0.013678,0.005107,-0.052585,-0.032091,0.023455,-0.039506,-0.071952,0.041092,0.008768,-0.012441,-0.021543,-0.013440,-0.011511,0.004086,-0.016868,-0.003797,0.062168,0.021020,-0.017684,-0.033057,...,0.052088,-0.036466,-0.048071,-0.044052,0.031582,-0.005607,0.046977,-0.046315,0.012168,-0.049222,0.055603,-0.008354,0.027261,0.011111,0.057126,0.008092,-0.002842,-0.010246,-0.037934,0.049191,-0.018213,0.045192,0.095404,-0.007928,-0.039156,0.048771,-0.034426,0.038492,0.039559,0.031308,0.052141,0.027275,-0.008246,0.043803,-0.032329,-0.021422,-0.018111,-0.009990,-0.060476,0.027701
4,-0.003644,0.028737,-0.052126,-0.000054,0.008107,-0.017344,-0.013791,0.012748,-0.035662,0.039845,0.006583,0.042427,0.022603,-0.026439,-0.011025,-0.032833,-0.010979,0.014916,0.026037,0.021582,0.004839,0.028031,-0.069568,-0.010970,0.035693,-0.046544,-0.054415,0.052412,0.012303,-0.015019,-0.008838,-0.008885,0.001337,-0.009247,-0.010716,-0.001443,0.034402,0.002758,-0.020472,-0.038950,...,0.054639,-0.058288,-0.054549,-0.024948,0.033424,-0.003804,0.013218,-0.054962,0.012327,-0.076053,0.053361,-0.018637,0.001523,-0.005087,0.054736,-0.002094,-0.014366,0.003204,-0.057732,0.036873,-0.031804,0.027221,0.055413,0.007639,-0.042472,0.022434,-0.037340,0.041808,0.0

In [15]:
y = embedding_df['label']
y.head()

0    1
1    0
2    1
3    1
4    0
Name: label, dtype: object

In [16]:
from sklearn.model_selection import train_test_split
# split into 70:30 ration
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 32)

In [17]:
y_train = [int(x) for x in y_train]
y_test = [int(x) for x in y_test]

In [18]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

In [19]:
# Create an SelectKBest object to select features with two best ANOVA F-Values
fvalue_selector = SelectKBest(f_classif, k=300)

# Apply the SelectKBest object to the features and target
X_kbest_train = fvalue_selector.fit_transform(X_train, y_train)

In [20]:
# Create an SelectKBest object to select features with two best ANOVA F-Values
fvalue_selector = SelectKBest(f_classif, k=300)

# Apply the SelectKBest object to the features and target
X_kbest_test = fvalue_selector.fit_transform(X_test, y_test)

In [21]:
from sklearn.linear_model import LogisticRegression

In [31]:
clf = LogisticRegression(random_state=0, C=100, penalty='l2')

In [32]:
#Fitting the model in the training data
clf.fit(X_kbest_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=100, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [33]:
#predicting the model
y_predict = clf.predict(X_kbest_test)

In [34]:
#accuracy score of the model
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_predict, normalize=True, sample_weight=None)

0.648508430609598

In [26]:
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import classification_report

In [27]:
result_svm = confusion_matrix(y_test, y_predict)
result_svm

array([[ 16, 312],
       [  4, 439]])

In [28]:
from sklearn.model_selection import GridSearchCV 
param_grid = {'penalty': ['l1','l2'], 
              'C': [0.001,0.01,0.1,1,10,100,1000]}

grid = GridSearchCV(LogisticRegression(), param_grid, refit = True, verbose = 3)
grid.fit(X_kbest_train, y_train)

Fitting 5 folds for each of 14 candidates, totalling 70 fits
[CV] C=0.001, penalty=l1 .............................................
[CV] ................... C=0.001, penalty=l1, score=nan, total=   0.0s
[CV] C=0.001, penalty=l1 .............................................
[CV] ................... C=0.001, penalty=l1, score=nan, total=   0.0s
[CV] C=0.001, penalty=l1 .............................................
[CV] ................... C=0.001, penalty=l1, score=nan, total=   0.0s
[CV] C=0.001, penalty=l1 .............................................
[CV] ................... C=0.001, penalty=l1, score=nan, total=   0.0s
[CV] C=0.001, penalty=l1 .............................................
[CV] ................... C=0.001, penalty=l1, score=nan, total=   0.0s
[CV] C=0.001, penalty=l2 .............................................
[CV] ................. C=0.001, penalty=l2, score=0.597, total=   0.0s
[CV] C=0.001, penalty=l2 .............................................
[CV] ...........

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The sc

[CV] C=0.1, penalty=l1 ...............................................
[CV] ..................... C=0.1, penalty=l1, score=nan, total=   0.0s
[CV] C=0.1, penalty=l1 ...............................................
[CV] ..................... C=0.1, penalty=l1, score=nan, total=   0.0s
[CV] C=0.1, penalty=l1 ...............................................
[CV] ..................... C=0.1, penalty=l1, score=nan, total=   0.0s
[CV] C=0.1, penalty=l1 ...............................................
[CV] ..................... C=0.1, penalty=l1, score=nan, total=   0.0s
[CV] C=0.1, penalty=l1 ...............................................
[CV] ..................... C=0.1, penalty=l1, score=nan, total=   0.0s
[CV] C=0.1, penalty=l2 ...............................................
[CV] ................... C=0.1, penalty=l2, score=0.597, total=   0.0s
[CV] C=0.1, penalty=l2 ...............................................
[CV] ................... C=0.1, penalty=l2, score=0.594, total=   0.0s
[CV] C

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)


[CV] .................... C=10, penalty=l2, score=0.619, total=   0.1s
[CV] C=10, penalty=l2 ................................................
[CV] .................... C=10, penalty=l2, score=0.592, total=   0.1s
[CV] C=10, penalty=l2 ................................................
[CV] .................... C=10, penalty=l2, score=0.602, total=   0.1s
[CV] C=10, penalty=l2 ................................................
[CV] .................... C=10, penalty=l2, score=0.624, total=   0.1s
[CV] C=100, penalty=l1 ...............................................
[CV] ..................... C=100, penalty=l1, score=nan, total=   0.0s
[CV] C=100, penalty=l1 ...............................................
[CV] ..................... C=100, penalty=l1, score=nan, total=   0.0s
[CV] C=100, penalty=l1 ...............................................
[CV] ..................... C=100, penalty=l1, score=nan, total=   0.0s
[CV] C=100, penalty=l1 ...............................................
[CV] .

...

[CV] .................. C=1000, penalty=l2, score=0.631, total=   0.1s
[CV] C=1000, penalty=l2 ..............................................
[CV] .................. C=1000, penalty=l2, score=0.636, total=   0.1s
[CV] C=1000, penalty=l2 ..............................................


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


[CV] .................. C=1000, penalty=l2, score=0.621, total=   0.1s
[CV] C=1000, penalty=l2 ..............................................
[CV] .................. C=1000, penalty=l2, score=0.641, total=   0.1s


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
[Parallel(n_jobs=1)]: Done  70 out of  70 | elapsed:    2.2s finished
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://

GridSearchCV(cv=None, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
                         'penalty': ['l1', 'l2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [30]:
 
# print best parameter after tuning 
print(grid.best_params_) 
  
# print how our model looks after hyper-parameter tuning 
print(grid.best_estimator_) 

{'C': 100, 'penalty': 'l2'}
LogisticRegression(C=100, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


In [ ]:
a, b, m = training(words, 100, 5)

In [ ]:
a, b, m = training(words, 25, 5)

For size = 25


/home/vanshika/anaconda3/envs/powerai_3.6_env/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  if sys.path[0] == '':


Result of UMNSRS-Sim data SpearmanrResult(correlation=0.1291200397469217, pvalue=0.021067703429049758)
Result of UMNSRS-Rel data SpearmanrResult(correlation=0.08940294960245106, pvalue=0.10658638479417483)
------------------------------------------------------------------


In [ ]:
a, b, m = training(words, 70, 5)

For size = 70
137677
137677
Result of UMNSRS-Sim data SpearmanrResult(correlation=0.4816698925491659, pvalue=1.2637866509260601e-24)
Result of UMNSRS-Rel data SpearmanrResult(correlation=0.4541854408839662, pvalue=7.220722015911838e-23)
------------------------------------------------------------------


/home/vanshika/anaconda3/envs/powerai_3.6_env/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  


In [ ]:
a, b, m = training(words, 50, 5)

For size = 50
137677
137677
Result of UMNSRS-Sim data SpearmanrResult(correlation=0.48417099560410753, pvalue=6.717386867801877e-25)
Result of UMNSRS-Rel data SpearmanrResult(correlation=0.45432781253696747, pvalue=6.975634824701234e-23)
------------------------------------------------------------------


/home/vanshika/anaconda3/envs/powerai_3.6_env/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  


In [ ]:
a, b, m = training(words, 150, 5)

For size = 150
137677
137677
Result of UMNSRS-Sim data SpearmanrResult(correlation=0.5096595538031236, pvalue=7.895418909344523e-28)
Result of UMNSRS-Rel data SpearmanrResult(correlation=0.48077578757408546, pvalue=8.55157938182282e-26)
------------------------------------------------------------------


/home/vanshika/anaconda3/envs/powerai_3.6_env/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  


In [ ]:
a, b, m = training(words, 250, 5)

For size = 250
137677
137677
Result of UMNSRS-Sim data SpearmanrResult(correlation=0.511479324157144, pvalue=4.771248116932873e-28)
Result of UMNSRS-Rel data SpearmanrResult(correlation=0.48851914499112725, pvalue=1.0728801855048895e-26)
------------------------------------------------------------------


/home/vanshika/anaconda3/envs/powerai_3.6_env/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  


In [ ]:
a, b, m = training(words, 550, 5)

For size = 550
137677
137677
Result of UMNSRS-Sim data SpearmanrResult(correlation=0.5156264787704207, pvalue=1.4968743528491977e-28)
Result of UMNSRS-Rel data SpearmanrResult(correlation=0.4947283291322275, pvalue=1.9538239363156295e-27)
------------------------------------------------------------------


/home/vanshika/anaconda3/envs/powerai_3.6_env/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  


In [ ]:
a, b, m = training(words, 1000, 5)

For size = 1000
137677
137677
Result of UMNSRS-Sim data SpearmanrResult(correlation=0.516083325782594, pvalue=1.316141725809059e-28)
Result of UMNSRS-Rel data SpearmanrResult(correlation=0.49958248025018603, pvalue=5.035120126054427e-28)
------------------------------------------------------------------


/home/vanshika/anaconda3/envs/powerai_3.6_env/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  


In [ ]:
a, b, m = training(words, 2000, 5)

For size = 2000
137677
137677
Result of UMNSRS-Sim data SpearmanrResult(correlation=0.5206364451995322, pvalue=3.6116837955897164e-29)
Result of UMNSRS-Rel data SpearmanrResult(correlation=0.5078675869533443, pvalue=4.7302161364263977e-29)
------------------------------------------------------------------


/home/vanshika/anaconda3/envs/powerai_3.6_env/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  


In [ ]:
a, b, m = training(words, 3000, 5)

For size = 3000


/home/vanshika/anaconda3/envs/powerai_3.6_env/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  if sys.path[0] == '':


Result of UMNSRS-Sim data SpearmanrResult(correlation=0.09418179455657628, pvalue=0.0930990853888878)
Result of UMNSRS-Rel data SpearmanrResult(correlation=0.1025049825694829, pvalue=0.06411297389515487)
------------------------------------------------------------------
